In [2]:
import warnings
warnings.filterwarnings("ignore")
import dbutils
import time
import pandas as pd
from IPython.display import display
import numpy as np
last_check = 0

In [5]:
#main_connector = dbutils.connect_to_main_database("dbconnection_dev.json")
#main_connector = dbutils.connect_to_main_database("dbconnection_qa.json")
main_connector = dbutils.connect_to_main_database("dbconnection_prod.json")

main_connector.get_all_tables()

Loading /data/home/tbehrens/notebooks/cgm-ml/cgm_database/dbconnection_prod.json...
Host:    cgm-in-bmz-postgresql-sql-prod.postgres.database.azure.com
DB-name: cgm


['pg_stat_statements',
 'pg_buffercache',
 'person',
 'measure',
 'artifact',
 'device',
 'model',
 'measure_result',
 'artifact_result',
 'result_transaction']

In [15]:
sql_statement = """
DROP VIEW IF EXISTS artifacts_with_targets;
CREATE VIEW artifacts_with_targets AS 
SELECT
    person.qr_code AS qr_code,
    artifact.id AS artifact_id,
    artifact.storage_path AS artifact_path,
    artifact.dataformat AS type,
    measure.age AS age,
    measure.height AS height,
    measure.weight AS weight,
    measure.muac AS muac,
    measure.head_circumference AS head_circumference
    --measure_quality.text_value AS status
    FROM artifact
    INNER JOIN measure ON artifact.measure_id=measure.id
    INNER JOIN person ON measure.person_id=person.id
    --INNER JOIN measure_quality ON measure_quality.measure_id=measure.id
    WHERE measure.height >= 60
    AND measure.height <= 120
    AND measure.weight >= 2
    AND measure.weight <= 20
    --AND measure_quality.key='expert_status'
    ;
            """
main_connector.execute(sql_statement)
print("Done.")

Done.


In [17]:
sql_statement = """
            SELECT artifact_path, qr_code, height, weight 
            FROM artifacts_with_targets 
            WHERE type='pcd'
            ;
            """
entries = main_connector.execute(sql_statement, fetch_all=True)
print("Found {} PCDs. Processing...".format(len(entries)))

Found 116298 PCDs. Processing...


In [19]:
sql_statement = """
            SELECT DISTINCT(qr_code)
            FROM artifacts_with_targets 
            WHERE type='pcd'
            ;
            """
entries = main_connector.execute(sql_statement, fetch_all=True)
print("Found {} QR codes. Processing...".format(len(entries)))

Found 3499 QR codes. Processing...


In [53]:
sql_statement = """
    SELECT 
        COUNT(*)
    FROM artifacts_with_targets 
    ;
    """
result = main_connector.execute(sql_statement, fetch_one=True)
print(len(result), result[0:2])

sql_statement = """
    SELECT 
        COUNT(*)
    FROM artifact
    WHERE type='pcd'
    ;
    """
result = main_connector.execute(sql_statement, fetch_one=True)
print(len(result), result)

sql_statement = """
    SELECT 
        COUNT(DISTINCT(measure_id))
    FROM artifact
    WHERE type='pcd'
    ;
    """
result = main_connector.execute(sql_statement, fetch_one=True)
print(len(result), result)

sql_statement = """
    SELECT 
        COUNT(*)
    FROM measure_quality
    ;
    """
result = main_connector.execute(sql_statement, fetch_one=True)
print(len(result), result)

1 (0,)
1 (13217,)
1 (430,)
1 (0,)


In [56]:
for file in ["dbconnection_dev.json", "dbconnection_qa.json", "dbconnection_prod.json"]:
    
    main_connector = dbutils.connect_to_main_database(file)
    print("")

    sql_statement = """
        SELECT COUNT(artifact)
          FROM artifact a
          INNER JOIN measure m ON a.measure_id=m.id
          WHERE a.type='pcd'
          AND m.type='manual'
        ;
        """
    result = main_connector.execute(sql_statement, fetch_one=True)
    print("PCDs with measure", result)

    sql_statement = """
        SELECT 
           to_timestamp(MIN(create_date / 1000))
        FROM artifact
        ;
        """
    result = main_connector.execute(sql_statement, fetch_one=True)
    print("OLDEST", result)

    sql_statement = """
        SELECT 
           to_timestamp(MAX(create_date / 1000))
        FROM artifact
        ;
        """
    result = main_connector.execute(sql_statement, fetch_one=True)
    print("NEWEST", result)
    
    print("")


Loading /data/home/tbehrens/notebooks/cgm-ml/cgm_database/dbconnection_dev.json...
Host:    cgm-dev.postgres.database.azure.com
DB-name: cgm_ml_db

PCDs with measure (146488,)
OLDEST (datetime.datetime(2018, 6, 29, 10, 40, 28, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),)
NEWEST (datetime.datetime(2019, 2, 8, 7, 4, 30, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),)

Loading /data/home/tbehrens/notebooks/cgm-ml/cgm_database/dbconnection_qa.json...
Host:    cgm-in-bmz-postgresql-sql-qa.postgres.database.azure.com
DB-name: cgm

PCDs with measure (13217,)
OLDEST (datetime.datetime(2019, 7, 16, 7, 27, 26, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),)
NEWEST (datetime.datetime(2019, 10, 30, 15, 1, 52, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),)

Loading /data/home/tbehrens/notebooks/cgm-ml/cgm_database/dbconnection_prod.json...
Host:    cgm-in-bmz-postgresql-sql-prod.postgres.database.azure.com
DB-name: cgm

PCDs with measu

In [13]:
sql_statement = """
    SELECT 
        COUNT(*)
    FROM artifacts_with_targets
    ;
    """
result = main_connector.execute(sql_statement, fetch_one=True)
print(len(result), result)

1 (0,)


In [ ]:
sql_statement = """
    SELECT 
        qr_code, 
        artifact_id, 
        REPLACE(artifact_path, '/whhdata', '/localssd/anondata/qrcode'),
        type, 
        age, 
        height, 
        weight, 
        muac, 
        head_circumference, 
        status 
    FROM artifacts_with_targets 
    WHERE type='pcd'
    ;
    """
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

In [ ]:
main_connector = dbutils.connect_to_main_database()

[WHH] Provide pointers to cleaner dataset  
sql_statement = """
    SELECT 
        qr_code, 
        artifact_id, 
        REPLACE(artifact_path, '/whhdata', '/localssd/anondata/qrcode'),
        type, 
        age, 
        height, 
        weight, 
        muac, 
        head_circumference, 
        status 
    FROM artifacts_with_targets 
    WHERE type='pcd'
    AND status='standing';
    """
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])



In [ ]:
!ls /localssd/anondata/qrcode-new/MP_WHH_2510/measurements/1540360822453/pc

In [ ]:
sql_statement = """
CREATE TABLE IF NOT EXISTS artifact_result (
    PRIMARY KEY(artifact_id, model_name, target_key),
    model_name TEXT NOT NULL,
    target_key TEXT NOT NULL,
    value REAL NOT NULL,
    artifact_id VARCHAR(255) REFERENCES artifact(id)
);
"""

#result = main_connector.execute(sql_statement)
#print(result)

In [ ]:
sql_statement = """
SELECT a.path, ABS(ar.value - m.height) AS loss 
  FROM artifact_result ar
  INNER JOIN artifact a ON ar.artifact_id=a.id
  INNER JOIN measure m ON a.measure_id=m.id
  INNER JOIN measure_quality mq ON mq.measure_id=m.id
  WHERE ar.target_key='height'
  AND ar.model_name ='20190913-1807_220-55height'
  AND m.type='manual'
--  AND ABS(ar.value - m.height) < 1.0
  AND mq.key='expert_status'
  AND mq.text_value='standing'
  ORDER BY loss
;
"""

result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result))
mean_loss = np.mean([x[1] for x in result])
print("MAE", mean_loss)
for x in result[0:10]:
    print(x)

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact;"
artifacts = main_connector.execute(sql_statement, fetch_one=True)[0]

sql_statement = "SELECT COUNT(*) FROM artifact_result;"
artifact_results = main_connector.execute(sql_statement, fetch_one=True)[0]

print(artifact_results, artifacts)
print(100 * artifact_results / artifacts)

In [ ]:
sql_statement = "SELECT DISTINCT(model_name) FROM artifact_result;"
result = main_connector.execute(sql_statement, fetch_one=True)[0]
print(result)


In [ ]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT height FROM measure m INNER JOIN person p on m.person_id=p.id WHERE qr_code='SAM-GOV-030' AND m.type='manual' ORDER BY m.timestamp;"
result = main_connector.execute(sql_statement, fetch_all=True)
result = [r[0] for r in result]
print(result)

In [ ]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT DISTINCT(type) FROM artifact_quality WHERE type='20190708-0919_2379-595height';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result)

In [ ]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT DISTINCT(type) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result)

In [ ]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE type='20190708-0919_2379-595height';"
result = main_connector.execute(sql_statement, fetch_one=True)[0]
print(last_check, result, "PCDs in artifact_quality")

if last_check != result:
    print("Change")
else:
    print("No change")
last_check = result

In [ ]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result, "PCDs in artifact")

# Scan level MAEs.

In [ ]:
main_connector = dbutils.connect_to_main_database()

sql_statement = ""
sql_statement += "SELECT COUNT(DISTINCT p.qr_code) FROM person p;"

result = main_connector.execute(sql_statement, fetch_one=True)[0]
print("Found {} qr-codes.".format(result))

In [ ]:
# Select all QR-code-create-date pairs. Those are scans.
sql_statement = ""
sql_statement += "SELECT DISTINCT p.qr_code, m.timestamp, m.id as measure_id"
sql_statement += " FROM measure m"
sql_statement += " INNER JOIN person p on m.person_id=p.id"
sql_statement += " WHERE m.type='manual'"
sql_statement += ";"

start = time.time()
scans = main_connector.execute(sql_statement, fetch_all=True)
end = time.time()
print(end - start)

print("Found {} manual scans".format(len(scans)))
#print(scans[200:300])



sql_statement = ""
sql_statement += "SELECT COUNT(*)"
sql_statement += " FROM artifact"
sql_statement += " WHERE type='pcrgb'"
sql_statement += ";"

scans = main_connector.execute(sql_statement, fetch_all=True)
print ("number of fused scans: " + str(scans))

sql_statement = ""
sql_statement += "SELECT COUNT(*)"
sql_statement += " FROM artifact"
sql_statement += " WHERE type='pcd'"
sql_statement += ";"

scans = main_connector.execute(sql_statement, fetch_all=True)
print ("number of pcd scans: " + str(scans))

In [ ]:
model = "20190708-0919_2379-595height"

for scan in scans[200:201]:
    qr_code = pair[0]
    create_date = pair[1]
    print(qr_code, create_date)
    
    sql_statement = ""
    sql_statement += "SELECT SUM(aq.value) "
    sql_statement += " FROM artifact_quality aq"
    sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
    sql_statement += " WHERE aq.key='mae'"
    sql_statement += " AND aq.type='{}'".format(model)
    sql_statement += " AND a.qr_code='{}'".format(qr_code)
    sql_statement += " AND a.create_date='{}'".format(create_date)
    sql_statement += ";"

    start = time.time()
    result = main_connector.execute(sql_statement, fetch_all=True)
    end = time.time()
    print(end - start)

In [ ]:
6697 * 10 / (60 * 60)

In [ ]:
print(qr_codes[0])
for qr_code in qr_codes[0:10]:
    qr_code = qr_code[0]
    
    # Get all create dates for that qr-code.
    sql_statement = ""
    sql_statement += "SELECT DISTINCT(a.create_date) FROM artifact a"
    sql_statement += " WHERE a.qr_code='{}'".format(qr_code)
    sql_statement += ";"
    
    start = time.time()
    create_dates = main_connector.execute(sql_statement, fetch_all=True)
    for create_date in create_dates:
        create_date = create_dates[0]
        print(r)
    end = time.time()
    print(end - start)
    break
    

In [ ]:
#SELECT a.qr_code, AVG(aq.value) 
#FROM artifact_quality as aq
#INNER JOIN artifact as a
#  ON a.artifact_id = aq.artifact_id
#WHERE aq.key='mae'
#AND aq.type='20190708-0919_2379-595height'
#GROUP BY a.qr_code, a.create_date;

#main_connector = dbutils.connect_to_main_database()

model_name = "20190708-0919_2379-595height"

sql_statement = ""
sql_statement += "SELECT a.qr_code, a.create_date, AVG(aq.value) FROM artifact_quality aq"
sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
sql_statement += " WHERE aq.key='mae'"
sql_statement += " AND aq.type='{}'".format(model_name)
sql_statement += " GROUP BY a.qr_code, a.create_date"
sql_statement += ";"

print(sql_statement)

start = time.time()
result = main_connector.execute(sql_statement, fetch_all=True)
end = time.time()
print(end - start)


print(len(result), "results")

#last_check = len(result)
rows = []
for r in result[0:10]:
    qr_code = r[0]
    create_date = r[1]
    mae = r[2]
    row = (qr_code, create_date, mae)
    print(row)
    rows.append(row)
    
columns = ["QR-Code", "Timestamp", "MAE"]

df = pd.DataFrame(rows, columns=columns)
display(df)

In [ ]:

sql_statement = "SELECT * FROM artifacts_with_targets WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='number_of_points';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)
print(result)

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact WHERE path LIKE '%scantimestamp%';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result)

sql_statement = "SELECT COUNT(*) FROM artifact WHERE path LIKE '% %';"
print(sql_statement)
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result)

sql_statement = "SELECT COUNT(*) FROM artifact WHERE path NOT LIKE '%scantimestamp%';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result)



In [ ]:
sql_statement = "SELECT id, tango_timestamp FROM artifact WHERE tango_timestamp = ROUND(tango_timestamp) AND type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

sql_statement = "SELECT id, tango_timestamp FROM artifact WHERE tango_timestamp != ROUND(tango_timestamp) AND type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)
print(len(result), result[0:2])

#for r in result[0:10]:
#    print(r[0], r[1])

In [ ]:
sql_statement = "SELECT id, tango_timestamp FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)
result = result[:100]
print(result[0:2])
for r in result:
    print(r[0], r[1])

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='confidence_avg';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifact quality KPIs")

sql_statement = "SELECT MIN(value) FROM artifact_quality WHERE key='number_of_points';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifact quality KPIs")

In [ ]:
main_connector = dbutils.connect_to_main_database()

sql_statement = "SELECT COUNT(*) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality")

sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE value=0.0;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality where value is 0.0")

sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='confidence_avg';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality with key confidence_avg")

sql_statement = "SELECT COUNT(*) FROM artifact_quality WHERE key='number_of_points';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "entries in artifact_quality with key number_of_points")

sql_statement = "SELECT COUNT(DISTINCT(artifact_id)) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "distinct artifact_ids in artifact_quality")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "PCDs in artifacts")

sql_statement = "SELECT COUNT(*) FROM artifacts_with_targets WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "PCDs in artifacts_with_targets")

In [ ]:
#sql_statement = "DELETE FROM artifact_quality WHERE value=0.0;"
#result = main_connector.execute(sql_statement, fetch_all=True)[0][0]

In [ ]:
sql_statement = "SELECT COUNT(*) FROM artifacts_with_targets WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "PCDs in artifacts_with_targets")